In [6]:
import os
import json
import pandas as pd
import numpy as np

def parchear_jsons_debug(directorio_raiz, ruta_db):
    if not os.path.exists(ruta_db):
        print(f"ERROR: No se encuentra la DB en {ruta_db}")
        return
    
    # 1. Consolidación rigurosa
    df_db = pd.read_json(ruta_db)
    # Limpieza de nombres para evitar fallos por espacios: 'TOI-122 b ' != 'TOI-122 b'
    df_db['pl_name'] = df_db['pl_name'].str.strip()
    df_db = df_db.groupby('pl_name').first().reset_index()
    
    print(f"DB cargada. Planetas únicos en DB: {len(df_db)}")

    mapeo = {
        "NEA_st_Teff": "st_teff", "NEA_st_rad": "st_rad",
        "NEA_st_mass": "st_mass", "NEA_ins_flux": "pl_insol",
        "NEA_pl_eq_temp": "pl_eqt", "NEA_a_au": "pl_orbsmax"
    }

    encontrados = 0
    for root, _, files in os.walk(directorio_raiz):
        for f in files:
            if f.endswith(".json"):
                encontrados += 1
                path = os.path.join(root, f)
                
                with open(path, 'r') as file:
                    data = json.load(file)
                
                # Acceso seguro a la estructura
                nea_sec = data.get("NEA", {})
                nombre_pl = nea_sec.get("NEA_pl_name", "").strip()
                
                if not nombre_pl:
                    print(f"Aviso: {f} no tiene 'NEA_pl_name'")
                    continue

                row = df_db[df_db['pl_name'] == nombre_pl]
                if row.empty:
                    # Si no lo encuentra, puede ser por el formato del nombre
                    continue
                
                info_db = row.iloc[0]
                modificado = False

                for loc_key, db_key in mapeo.items():
                    val_actual = nea_sec.get(loc_key)
                    val_nuevo = info_db.get(db_key)

                    # Evaluación lógica: actualiza si el dato local es nulo/NaN
                    # pero el dato en DB es un número real.
                    if (pd.isna(val_actual) or val_actual == "NaN") and pd.notna(val_nuevo):
                        data["NEA"][loc_key] = float(val_nuevo)
                        modificado = True

                if modificado:
                    with open(path, 'w') as file:
                        json.dump(data, file, indent=4)
                    print(f"Actualizado: {nombre_pl} en {f}")

    print(f"Proceso finalizado. Archivos .json analizados: {encontrados}")

# --- AJUSTA ESTAS RUTAS ---

ruta_base_datos = r"C:\Users\jorge\Desktop\TE+\tess\toi_data_test.json"
directorio_resultados = r"C:\Users\jorge\Desktop\TE+\tess\resultados" 

parchear_jsons_debug(directorio_resultados, ruta_base_datos)




DB cargada. Planetas únicos en DB: 354
Actualizado: TOI-1135 b en TOI-1135 b.json
Actualizado: TOI-1174 b en TOI-1174 b.json
Actualizado: TOI-1180 b en TOI-1180 b.json
Actualizado: TOI-1181 b en TOI-1181 b.json
Actualizado: TOI-1184 b en TOI-1184 b.json
Actualizado: TOI-1244 b en TOI-1244 b.json
Actualizado: TOI-1268 b en TOI-1268 b.json
Actualizado: TOI-1296 b en TOI-1296 b.json
Actualizado: TOI-1298 b en TOI-1298 b.json
Actualizado: TOI-1437 b en TOI-1437 b.json
Actualizado: TOI-1444 b en TOI-1444 b.json
Actualizado: TOI-157 b en TOI-157 b.json
Actualizado: TOI-1691 b en TOI-1691 b.json
Actualizado: TOI-1696 b en TOI-1696 b.json
Actualizado: TOI-1723 b en TOI-1723 b.json
Actualizado: TOI-1728 b en TOI-1728 b.json
Actualizado: TOI-172 b en TOI-172 b.json
Actualizado: TOI-1758 b en TOI-1758 b.json
Actualizado: TOI-1759 b en TOI-1759 b.json
Actualizado: TOI-1789 b en TOI-1789 b.json
Actualizado: TOI-1799 b en TOI-1799 b.json
Actualizado: TOI-1842 b en TOI-1842 b.json
Actualizado: TOI-21